In [3]:
import psycopg2
import pandas as pd
import sys
sys.path.append("../")
from src import soporte_wallapop as swallapop

He creado unas tablas vacías con la misma estructura que las tablas para introducir las que nos interese:

In [ ]:
CREATE TABLE wallapop_interesante AS
SELECT * FROM datos_wallpop_limpios WHERE 1=0;


CREATE TABLE vinted_interesante AS
SELECT * FROM datos_vinted_filtrados WHERE 1=0;



CREATE TABLE cash_converters_interesante AS
SELECT * FROM datos_cash_converters WHERE 1=0;



CREATE TABLE mercado_libre_interesante AS
SELECT * FROM datos_mercado_libre WHERE 1=0;

Decido hacer la media de venta de mi producto (ps5), para saber que productos me pueden interesar comprar

In [40]:
conn = psycopg2.connect(
    host="localhost",
    database="proyecto_5",
    user="postgres",
    password="admin"
)
cursor = conn.cursor()

In [12]:
conn.rollback()

## Wallapop

In [ ]:

cursor = conn.cursor()

media_productos_wallapop = """
SELECT AVG(precio) AS media_precio FROM datos_wallpop_limpios;
"""

cursor.execute(media_productos_wallapop)
media_precio = cursor.fetchall()[0]  # Obtener solo el valor de la media

print("Media de precio:", media_precio)

# Paso 3: Insertar en wallapop_interesante los productos que cumplen el criterio
consulta_insertar = """
INSERT INTO wallapop_interesante (wallpop_id, user_id, titulo, descripcion, categoria_id, precio, 
                                  imagenes, reservado, localizacion, golpe, reacondicionado, 
                                  url_producto, posibilidad_envio)
SELECT wallpop_id, user_id, titulo, descripcion, categoria_id, precio, 
       imagenes, reservado, localizacion, golpe, reacondicionado, 
       url_producto, posibilidad_envio
FROM datos_wallpop_limpios
WHERE precio < %s;
"""

# Ejecutar la consulta de inserción con la media como parámetro
cursor.execute(consulta_insertar, (media_precio,))

# Confirmar los cambios en la base de datos
conn.commit()

# Cerrar el cursor y la conexión
cursor.close()
conn.close()

print("Productos copiados a wallapop_interesante con éxito.")


In [56]:
# Obtener datos desde la base de datos
df_datos_wallapop = swallapop.obtener_datos_wallapop_sql()

C:\Users\almaz\AppData\Local\Temp\ipykernel_19092\2915255581.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [57]:
# Generar PDF a partir de los datos
archivo_pdf = "datos_wallapop_interesantes.pdf"
swallapop.generar_pdf(df_datos_wallapop, archivo_pdf)

In [53]:
# Enviar el PDF por correo
swallapop.enviar_correo_con_adjunto(
    destinatario="almazanlucas@gmail.com",
    asunto="Productos de Wallapop - PS5",
    cuerpo="Adjunto encontrarás el informe de productos de PS5 en Wallapop.",
    archivo_adj=archivo_pdf
)

Correo enviado exitosamente.


## Vinted

> A dia de hoy no hay Play Station 5 a la venta en Vinted, no obstante es interesante no perder de vista esta plataforma ya que puede haber en algun momento.

In [31]:
media_vinted = """
SELECT AVG(total_item_price) AS media_precio FROM datos_vinted_filtrados;
"""
cursor.execute(media_vinted)
media_precio_vinted = cursor.fetchone()[0]
print("Media de precio para Vinted:", media_precio_vinted)

insertar_vinted = """
INSERT INTO vinted_interesante (id, title, is_visible, currency, brand_title, url, 
                                promoted, favourite_count, is_favourite, total_item_price, status)
SELECT id, title, is_visible, currency, brand_title, url, 
       promoted, favourite_count, is_favourite, total_item_price, status
FROM datos_vinted_filtrados
WHERE total_item_price < %s;
"""
cursor.execute(insertar_vinted, (media_precio_vinted,))

Media de precio para Vinted: 11784.3250000000000000


In [ ]:
df_datos_wallapop = swallapop.obtener_datos_vinted_sql()

## Cash Converters

In [38]:
media_cash_converters = """
SELECT AVG(precio) AS media_precio FROM datos_cash_converters;
"""
cursor.execute(media_cash_converters)
media_precio_cash_converters = cursor.fetchone()[0]
print("Media de precio para Cash Converters:", media_precio_cash_converters)

# 2. Insertar los productos de Cash Converters con precio superior a la media en cash_converters_interesante
insertar_cash_converters = """
INSERT INTO cash_converters_interesante (titulo, descripcion, precio)
SELECT titulo, descripcion, precio
FROM datos_cash_converters
WHERE precio < %s;
"""
cursor.execute(insertar_cash_converters, (media_precio_cash_converters,))

# Confirmar los cambios en la base de datos
conn.commit()

# Cerrar el cursor y la conexión
cursor.close()
conn.close()

Media de precio para Cash Converters: 370.95


In [ ]:
429.6542253521126761

## Mercado Libre

In [41]:
media_mercado_libre = """
SELECT AVG(precio_eur) AS media_precio FROM datos_mercado_libre;
"""
cursor.execute(media_mercado_libre)
media_precio_mercado_libre = cursor.fetchone()[0]
print("Media de precio para Mercado Libre:", media_precio_mercado_libre)

insertar_mercado_libre = """
INSERT INTO mercado_libre_interesante (titulo, link, precio_eur)
SELECT titulo, link, precio_eur
FROM datos_mercado_libre
WHERE precio_eur < %s;
"""
cursor.execute(insertar_mercado_libre, (media_precio_mercado_libre,))

Media de precio para Mercado Libre: 449.7244897959183673
